In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../") # go to parent dir
from src.data.diversity_document import DiversityDocument
import spacy
nlp = spacy.load('en')
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim

df = pd.read_csv('../data/external/2017-18stats.csv')
with open('../data/external/download_whitelist.txt', 'r') as f:
    whitelist = [line.strip() for line in f]
df['CompanyLinkToGPGInfo'] = df['CompanyLinkToGPGInfo'].astype(str)

number_of_companies = len(df.index)
texts = list()
metadata = list()

In [2]:
for index, row in df.head(2000).iterrows():
    percent_complete = ((index + 1) / number_of_companies) * 100
    print('%.2f percent complete' % (percent_complete))

    doc = DiversityDocument(row)

    if doc.url_hash in whitelist:
        continue

    if doc.raw_text is not None:
        texts.append(doc.raw_text)
        metadata.append(row.to_dict())

    if not doc.has_downloaded_file and not doc.company_link == 'nan':
        doc.download()
        if doc.raw_text is None:
            with open('../data/external/download_whitelist.txt', 'a') as whitelist_file:
                whitelist_file.write(doc.url_hash + '\n')

0.01 percent complete
0.02 percent complete
0.03 percent complete
0.04 percent complete
0.05 percent complete
0.06 percent complete
0.07 percent complete
0.08 percent complete
0.09 percent complete
0.10 percent complete
0.11 percent complete
0.12 percent complete
0.13 percent complete
0.14 percent complete
0.15 percent complete
0.16 percent complete
0.17 percent complete
0.18 percent complete
0.19 percent complete
0.20 percent complete
0.21 percent complete
0.22 percent complete
0.23 percent complete
0.24 percent complete
0.25 percent complete
0.26 percent complete
0.27 percent complete
0.28 percent complete
0.29 percent complete
0.30 percent complete
0.31 percent complete
0.31 percent complete
0.32 percent complete
0.33 percent complete
0.34 percent complete
0.35 percent complete
0.36 percent complete
0.37 percent complete
0.38 percent complete
0.39 percent complete
0.40 percent complete
0.41 percent complete
0.42 percent complete
0.43 percent complete
0.44 percent complete
0.45 perce

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [3]:
processed_docs = []
for doc in nlp.pipe(texts, n_threads=2, batch_size=8):
    # Process document using Spacy NLP pipeline.

    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)

docs = processed_docs
del processed_docs


In [4]:
# Add bigrams and trigrams to docs (only ones that appear 5 times or more).
bigram = Phrases(docs, min_count=5)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)


# Create a dictionary representation of the documents, and filter out frequent and rare words.
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 20
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

_ = dictionary[0]  # This sort of "initializes" dictionary.id2token.


/Users/mrmattwright/.local/share/virtualenvs/gender-pay-gap-nlp-5378hxDS/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [5]:

# Vectorize data.
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 3496
Number of documents: 1274


In [24]:
lda_model = LdaModel(corpus=corpus,
               id2word=dictionary,
               num_topics=12, 
               random_state=100,
               update_every=1,
               chunksize=100,
               passes=10,
               alpha='auto',
               per_word_topics=True)

In [25]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/Users/mrmattwright/.local/share/virtualenvs/gender-pay-gap-nlp-5378hxDS/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      21.643629        1       1 -0.203701  0.143251
1      16.292107        1       2 -0.179684 -0.487891
8      15.425691        1       3  0.163978  0.023973
6      12.988004        1       4 -0.076651  0.142345
10      7.322340        1       5  0.065220  0.064985
5       5.867578        1       6  0.021120  0.083067
2       5.167725        1       7  0.185018 -0.011287
7       3.794480        1       8  0.131403 -0.015804
3       3.640553        1       9 -0.265458  0.095276
4       2.820217        1      10  0.178060 -0.035236
9       2.645020        1      11 -0.172387  0.065622
11      2.392666        1      12  0.153080 -0.068300, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
1687  Default  4564.000000               o  4564.000000  30.0000  30.0000
1660  Default  4560.000000               g  4560.000000  29.0000  29.0000
1478  Default  4150.000000               c  4150.000000  28.0000  28.0000
758   Default  4113.000000               s  4113.000000  27.0000  27.0000
2173  Default  4085.000000               w  4085.000000  26.0000  26.0000
1591  Default  4050.000000               y  4050.000000  25.0000  25.0000
1061  Default  4028.000000               r  4028.000000  24.0000  24.0000
2682  Default  3979.000000               n  3979.000000  23.0000  23.0000
1212  Default  3979.000000               f  3979.000000  22.0000  22.0000
524   Default  3975.000000               q  3975.000000  21.0000  21.0000
2833  Default  3963.000000               u  3963.000000  20.0000  20.0000
1029  Default  3993.000000               m  3993.000000  19.0000  19.0000
2835  Default  3901.000000               v  3901.000000  18.0000  18.0000
2009  Default  3878.000000               k  3878.000000  17.0000  17.0000
663   Default  3801.000000               h  3801.000000  16.0000  16.0000
175   Default  4718.000000           bonus  4718.000000  15.0000  15.0000
1118  Default  3820.000000               x  3820.000000  14.0000  14.0000
2842  Default  3918.000000               z  3918.000000  13.0000  13.0000
2808  Default  3575.000000               j  3575.000000  12.0000  12.0000
1045  Default  3538.000000               p  3538.000000  11.0000  11.0000
548   Default  3791.000000               t  3791.000000  10.0000  10.0000
2815  Default  3515.000000               l  3515.000000   9.0000   9.0000
1485  Default  3776.000000               d  3776.000000   8.0000   8.0000
373   Default  4310.000000               e  4310.000000   7.0000   7.0000
1847  Default  3405.000000               b  3405.000000   6.0000   6.0000
227   Default  3405.000000        quartile  3405.000000   5.0000   5.0000
333   Default  4250.000000          female  4250.000000   4.0000   4.0000
246   Default  4751.000000           woman  4751.000000   3.0000   3.0000
338   Default  3173.000000            male  3173.000000   2.0000   2.0000
599   Default  1366.000000            care  1366.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
1442  Topic12    34.589443       quarterly    39.977055   3.5880  -6.1066
3033  Topic12    55.353416        releases    68.659752   3.5173  -5.6364
1285  Topic12   165.584763       investors   207.081879   3.5091  -4.5407
423   Topic12    48.595989    tax_strategy    61.613045   3.4954  -5.7666
2614  Topic12   103.878822        advisory   142.148895   3.4191  -5.0070
616   Topic12   471.367950       corporate   843.475830   3.1509  -3.4945
1283  Topic12   274.621338      governance   455.232056   3.2273  -4.0348
472   Topic12   458.745209       financial   835.558960   3.1332  -3.5217
701   Topic12   288.763336           media   501.456299   3.1809  -3.9846
544   Topic12   239.396973         slavery   406.873108   3.2024  -4.1721
1612  Topic12   125